# Hello World

In [2]:
# !python -m pip install -U crewai crewai-tools

In [47]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = "XX"
os.environ["OPENAI_API_KEY"] = "Your Key"


In [4]:
# Instantiate tools
docs_tool = DirectoryReadTool(directory='./')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()


/Users/sinanozdemir/.pyenv/versions/3.12.3/lib/python3.12/site-packages/alembic/config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [5]:
docs_tool

DirectoryReadTool(name='List files in directory', description="Tool Name: List files in directory\nTool Arguments: {}\nTool Description: A tool that can be used to list ./'s content.", env_vars=[], args_schema=<class 'crewai_tools.tools.directory_read_tool.directory_read_tool.FixedDirectoryReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x13a16fc40>, result_as_answer=False, max_usage_count=None, current_usage_count=0, directory='./')

In [6]:
search_tool.run(search_query='sinan ozdemir')

Using Tool: Search the internet with Serper


{'searchParameters': {'q': 'sinan ozdemir',
  'type': 'search',
  'num': 10,
  'engine': 'google'},
 'knowledgeGraph': {'title': 'Sinan Ozdemir',
  'type': 'Author',
  'website': 'https://sinanozdemir.ai/',
  'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQD00-MCEVfGiW4GQMN9-VZk4SfeqGXAE2OP6pt0IE&s=0',
  'description': 'Sinan Ozdemir is a data scientist, start-up founder, and educator living in the San Francisco Bay Area. He studied pure mathematics at the Johns Hopkins University. ...',
  'descriptionSource': 'Google Books',
  'descriptionLink': 'https://books.google.com/books/about/Feature_Engineering_Made_Easy.html?id=rNRJDwAAQBAJ&source=kp_author_description',
  'attributes': {}},
 'organic': [{'title': 'Sinan Ozdemir - LoopGenius - LinkedIn',
   'link': 'https://www.linkedin.com/in/sinan-ozdemir',
   'snippet': 'Specialized in areas including time management as well as mathematics and computer science logistics.',
   'position': 1},
  {'title': 'Sinan Ozdemir',

In [7]:
# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst\'s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)


In [40]:
# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
crew.kickoff()

🚀 Crew: crew
├── 📋 Task: 079be7ac-487f-4a52-b21c-2239bdeee149
│   Assigned to: Task Execution Planner
│   Status: ✅ Completed
├── 📋 Task: 1b78012e-febe-4498-9650-654f59ed0ecc
│   Assigned to: Market Research Analyst
│   Status: ✅ Completed
│   └── 🔧 Used Search the internet with Serper (1)
└── 📋 Task: a34ca59e-548b-472b-bf87-726ef4cddde4
    Status: Executing Task...
    ├── 🔧 Used List files in directory (1)
    ├── 🔧 Used Read a file's content (1)
    ├── 🔧 Used Read a file's content (2)
    ├── 🔧 Used Read a file's content (3)
    ├── 🔧 Used Read a file's content (4)
    ├── 🔧 Used Read a file's content (5)
    ├── 🔧 Used Read a file's content (6)
    ├── 🔧 Used Read a file's content (7)
    ├── 🔧 Used Read a file's content (8)
    ├── 🔧 Used Read a file's content (9)
    ├── 🔧 Used Read a file's content (10)
    ├── 🔧 Used Read a file's content (11)
    └── 🔧 Used Read a file's content (12)

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Transformative Trends in the AI Industry                                                                 │
│                                                                                                                 │
│  The AI industry is currently experiencing a seismic shift, fueled by the explosive growth of generative AI.    │
│  This revolutionary technology enables machines to produce creative content, from art and music to text and     │
│  design, challenging traditional notions of creativity. Generative AI not only enhances productivity in         │
│  content creation but also brings innovative business solutions to the forefront. Companies are harnessing      │
│  this technology to automate tasks, personalize customer experiences, and optimize operational efficiencies,    │
│  marking a significant departure from the AI applications of the past.                                          │
│                                                                                                                 │
│  In parallel, advancements in Natural Language Processing (NLP) continue to revolutionize how businesses        │
│  interact with customers and stakeholders. With improved language models, AI systems can understand and         │
│  generate human-like responses, leading to seamless communications. This evolution is transforming customer     │
│  service, making automated engagements more efficient and human-like. As we embrace these advancements,         │
│  ethical considerations regarding data usage and language biases come into play, necessitating a thoughtful     │
│  approach to implementation.                                                                                    │
│                                                                                                                 │
│  Moreover, the increasing integration of AI across various industries is paving the way for innovation while    │
│  reshaping business landscapes. From healthcare and finance to manufacturing and entertainment, AI              │
│  technologies are enhancing decision-making processes and driving strategic transformations. Businesses that    │
│  recognize the potential of AI to revolutionize their operations are likely to remain competitive in an         │
│  ever-evolving market. This trend emphasizes the importance of adaptability and forward-thinking in the face    │
│  of rapid technological change.                                                                                 │
│                                                                                                                 │
│  As we move into a future heavily influenced by AI, it is crucial to navigate both the opportunities and        │
│  challenges that arise. Companies must be prepared to address ethical considerations, ensure responsible AI     │
│  usage, and remain vigilant about the implications of these technologies. The path forward is not just about    │
│  leveraging AI for efficiency but rather fostering an environment where innovation can thrive responsibly.      │
│  Embracing these trends today will lay the groundwork for a smarter, more connected tomorrow.                   │
│  ```                                                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a34ca59e-548b-472b-bf87-726ef4cddde4                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ffede9b8-6444-462f-abaa-e8c950b58dd1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # The Transformative Trends in the AI Industry                                                                 │
│                                                                                                                 │
│  The AI industry is currently experiencing a seismic shift, fueled by the explosive growth of generative AI.    │
│  This revolutionary technology enables machines to produce creative content, from art and music to text and     │
│  design, challenging traditional notions of creativity. Generative AI not only enhances productivity in         │
│  content creation but also brings innovative business solutions to the forefront. Companies are harnessing      │
│  this technology to automate tasks, personalize customer experiences, and optimize operational efficiencies,    │
│  marking a significant departure from the AI applications of the past.                                          │
│                                                                                                                 │
│  In parallel, advancements in Natural Language Processing (NLP) continue to revolutionize how businesses        │
│  interact with customers and stakeholders. With improved language models, AI systems can understand and         │
│  generate human-like responses, leading to seamless communications. This evolution is transforming customer     │
│  service, making automated engagements more efficient and human-like. As we embrace these advancements,         │
│  ethical considerations regarding data usage and language biases come into play, necessitating a thoughtful     │
│  approach to implementation.                                                                                    │
│                                                                                                                 │
│  Moreover, the increasing integration of AI across various industries is paving the way for innovation while    │
│  reshaping business landscapes. From healthcare and finance to manufacturing and entertainment, AI              │
│  technologies are enhancing decision-making processes and driving strategic transformations. Businesses that    │
│  recognize the potential of AI to revolutionize their operations are likely to remain competitive in an         │
│  ever-evolving market. This trend emphasizes the importance of adaptability and forward-thinking in the face    │
│  of rapid technological change.                                                                                 │
│                                                                                                                 │
│  As we move into a future heavily influenced by AI, it is crucial to navigate both the opportunities and        │
│  challenges that arise. Companies must be prepared to address ethical considerations, ensure responsible AI     │
│  usage, and remain vigilant about the implications of these technologies. The path forward is not just about    │
│  leveraging AI for efficiency but rather fostering an environment where innovation can thrive responsibly.      │
│  Embracing these trends today will lay the groundwork 

CrewOutput(raw='```markdown\n# The Transformative Trends in the AI Industry\n\nThe AI industry is currently experiencing a seismic shift, fueled by the explosive growth of generative AI. This revolutionary technology enables machines to produce creative content, from art and music to text and design, challenging traditional notions of creativity. Generative AI not only enhances productivity in content creation but also brings innovative business solutions to the forefront. Companies are harnessing this technology to automate tasks, personalize customer experiences, and optimize operational efficiencies, marking a significant departure from the AI applications of the past.\n\nIn parallel, advancements in Natural Language Processing (NLP) continue to revolutionize how businesses interact with customers and stakeholders. With improved language models, AI systems can understand and generate human-like responses, leading to seamless communications. This evolution is transforming customer se

In [41]:
# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=False,  # Enable planning feature
)

# Execute tasks
crew.kickoff()

🚀 Crew: crew
├── 📋 Task: 1b78012e-febe-4498-9650-654f59ed0ecc
│   Assigned to: Market Research Analyst
│   Status: ✅ Completed
│   └── 🔧 Used Search the internet with Serper (2)
└── 📋 Task: a34ca59e-548b-472b-bf87-726ef4cddde4
    Status: Executing Task...
    ├── 🔧 Used List files in directory (2)
    ├── 🔧 Used Read a file's content (13)
    ├── 🔧 Used List files in directory (3)
    ├── 🔧 Used Read a file's content (14)
    ├── 🔧 Used Read a file's content (15)
    ├── 🔧 Used List files in directory (4)
    ├── 🔧 Used Read a file's content (16)
    ├── 🔧 Used List files in directory (5)
    ├── 🔧 Used Read a file's content (17)
    ├── 🔧 Used Read a file's content (18)
    ├── 🔧 Used Read a file's content (19)
    ├── 🔧 Used Read a file's content (20)
    ├── 🔧 Used Read a file's content (21)
    ├── 🔧 Used Read a file's content (22)
    ├── 🔧 Used Read a file's content (23)
    ├── 🔧 Used Read a file's content (24)
    ├── 🔧 Used Read a file's content (25)
    └── 🔧 Used List files in directory (6)

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Due to the inability to retrieve content from existing blog posts, I will create a new engaging blog post      │
│  about the AI industry based on the latest trends provided. Here is the blog post formatted in markdown:        │
│                                                                                                                 │
│  # Exploring the Transformative Trends in the AI Industry                                                       │
│                                                                                                                 │
│  Artificial Intelligence (AI) is no longer a futuristic concept; it’s transforming industries and shaping our   │
│  day-to-day experiences in unprecedented ways. The latest trends indicate a robust growth trajectory for the    │
│  AI market, with projections expecting it to surge from **$189 billion** in 2023 to **$4.8 trillion** by 2033.  │
│  This exponential growth underscores AI's vital role in enhancing economic competitiveness and efficiency       │
│  across various sectors.                                                                                        │
│                                                                                                                 │
│  One of the most remarkable advancements in the AI landscape is the rise of **generative AI**, which has        │
│  gained traction as a game-changer in content creation and user interaction. According to the **2023 McKinsey   │
│  Global Survey**, generative AI tools not only enhance productivity but also foster creativity by providing     │
│  innovative solutions that automate previously labor-intensive tasks. Businesses are leveraging these tools to  │
│  personalize experiences for users, thereby improving engagement and satisfaction.                              │
│                                                                                                                 │
│  Additionally, we see a shift towards **industry-specific AI solutions** that cater to the unique needs of      │
│  various sectors. This trend is characterized by a strong emphasis on **Explainable AI (XAI)**, which aims to   │
│  make AI decision-making processes more transparent. Stakeholders are placing high value on explainability to   │
│  foster trust and ensure ethical AI use. Companies are adopting tailored AI solutions that align with their     │
│  operational demands while prioritizing the need for transparency.                                              │
│                                                                                                                 │
│  As AI continues to evolve, its implications are vast and significant. With the ongoing investment in AI        │
│  technologies and the continuous development of specialized AI systems, the future promises exciting            │
│  opportunities. The collective efforts in harnessing AI’s potential may pave the way for remarkable             │
│  innovations that could redefine productivity, creativity, and collective learning across industries. Now is    │
│  the time for businesses and individuals alike to embrace this technological revolution and participate in      │
│  shaping the AI-driven future.                                                                                  │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a34ca59e-548b-472b-bf87-726ef4cddde4                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f3cdf889-a66c-4632-8bba-447fe3ae63d5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Due to the inability to retrieve content from existing blog posts, I will create a new engaging  │
│  blog post about the AI industry based on the latest trends provided. Here is the blog post formatted in        │
│  markdown:                                                                                                      │
│                                                                                                                 │
│  # Exploring the Transformative Trends in the AI Industry                                                       │
│                                                                                                                 │
│  Artificial Intelligence (AI) is no longer a futuristic concept; it’s transforming industries and shaping our   │
│  day-to-day experiences in unprecedented ways. The latest trends indicate a robust growth trajectory for the    │
│  AI market, with projections expecting it to surge from **$189 billion** in 2023 to **$4.8 trillion** by 2033.  │
│  This exponential growth underscores AI's vital role in enhancing economic competitiveness and efficiency       │
│  across various sectors.                                                                                        │
│                                                                                                                 │
│  One of the most remarkable advancements in the AI landscape is the rise of **generative AI**, which has        │
│  gained traction as a game-changer in content creation and user interaction. According to the **2023 McKinsey   │
│  Global Survey**, generative AI tools not only enhance productivity but also foster creativity by providing     │
│  innovative solutions that automate previously labor-intensive tasks. Businesses are leveraging these tools to  │
│  personalize experiences for users, thereby improving engagement and satisfaction.                              │
│                                                                                                                 │
│  Additionally, we see a shift towards **industry-specific AI solutions** that cater to the unique needs of      │
│  various sectors. This trend is characterized by a strong emphasis on **Explainable AI (XAI)**, which aims to   │
│  make AI decision-making processes more transparent. Stakeholders are placing high value on explainability to   │
│  foster trust and ensure ethical AI use. Companies are adopting tailored AI solutions that align with their     │
│  operational demands while prioritizing the need for transparency.                                              │
│                                                                                                                 │
│  As AI continues to evolve, its implications are vast and significant. With the ongoing investment in AI        │
│  technologies and the continuous development of specialized AI systems, the future promises exciting            │
│  opportunities. The collective efforts in harnessing AI’s potential may pave the way for remarkable             │
│  innovations that could redefine productivity, creativity, and collective learning across industries. Now is    │
│  the time for businesses and individuals alike to embr

CrewOutput(raw="Due to the inability to retrieve content from existing blog posts, I will create a new engaging blog post about the AI industry based on the latest trends provided. Here is the blog post formatted in markdown:\n\n# Exploring the Transformative Trends in the AI Industry\n\nArtificial Intelligence (AI) is no longer a futuristic concept; it’s transforming industries and shaping our day-to-day experiences in unprecedented ways. The latest trends indicate a robust growth trajectory for the AI market, with projections expecting it to surge from **$189 billion** in 2023 to **$4.8 trillion** by 2033. This exponential growth underscores AI's vital role in enhancing economic competitiveness and efficiency across various sectors.\n\nOne of the most remarkable advancements in the AI landscape is the rise of **generative AI**, which has gained traction as a game-changer in content creation and user interaction. According to the **2023 McKinsey Global Survey**, generative AI tools no

In [9]:
from crewai import Process
data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question.',
    backstory='You are a data expert',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)

test_task = Task(
    description="Understand the topic of Natural Language Processing and summarize it for me",
    agent=data_analyst,
    expected_output='I want the response to be as short as possible.'
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
)

output = crew.kickoff()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Task: Understand the topic of Natural Language Processing and summarize it for me                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Natural Language Processing overview 2023\"}"                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'Natural Language Processing overview 2023', 'type': 'search', 'num': 10,           │
│  'engine': 'google'}, 'organic': [{'title': '[PDF] NaturalLanguage Processing:AnOverview - SCIREA', 'link':     │
│  'https://article.scirea.org/pdf/120314.pdf', 'snippet': 'This paper provides a comprehensive overview of NLP,  │
│  tracing its evolution from its inception to its current state-of-the-art methodologies ...', 'position': 1},   │
│  {'title': 'What Is NLP (Natural Language Processing)? - IBM', 'link':                                          │
│  'https://www.ibm.com/think/topics/natural-language-processing', 'snippet': 'NLP enables computers and digital  │
│  devices to recognize, understand and generate text and speech by combining computational linguistics, the      │
│  rule ...', 'position': 2}, {'title': 'Natural Language Processing: An Overview - SCIREA', 'link':              │
│  'https://www.scirea.org/journal/PaperInformation?PaperID=9921', 'snippet': 'This paper provides a              │
│  comprehensive overview of NLP, tracing its evolution from its inception to its current state-of-the-art        │
│  methodologies. At ...', 'position': 3}, {'title': 'Our Guide to Natural Language Processing, an Introduction   │
│  to NLP', 'link': 'https://kili-technology.com/data-labeling/nlp', 'snippet': 'In this blog, we will dive into  │
│  the basics of NLP, how it works, its history and research, different NLP tasks, including the rise of large    │
│  language models ( ...', 'position': 4}, {'title': 'Natural Language Processing (NLP) [A Complete Guide]',      │
│  'link': 'https://www.deeplearning.ai/resources/natural-language-processing/', 'snippet': 'Natural Language     │
│  Processing is the discipline of building machines that can manipulate language in the way that it is written,  │
│  spoken, ...', 'position': 5}, {'title': 'Natural language processing capabilities and use cases - the 2023     │
│  ...', 'link':                                                                                                  │
│  'https://fastdatascience.com/natural-language-processing/natural-language-processing-capabilities-and-use-cas  │
│  es/', 'snippet': 'In this article we will discuss what NLP is, specific NLP capabilities and use cases, ...    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Thought: Thought: I will gather more detailed information on Natural Language Processing from reliable         │
│  sources.                                                                                                       │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'Natural Language Processing definition and applications 2023', 'type': 'search',   │
│  'num': 10, 'engine': 'google'}, 'organic': [{'title': 'What Is NLP (Natural Language Processing)? - IBM',      │
│  'link': 'https://www.ibm.com/think/topics/natural-language-processing', 'snippet': 'NLP enables computers and  │
│  digital devices to recognize, understand and generate text and speech by combining computational linguistics,  │
│  the rule-based modeling of human language together with statistical modeling, machine learning and deep        │
│  learning.', 'position': 1}, {'title': 'Natural Language Processing (NLP) [A Complete Guide]', 'link':          │
│  'https://www.deeplearning.ai/resources/natural-language-processing/', 'snippet': 'Natural Language Processing  │
│  is the discipline of building machines that can manipulate language in the way that it is written, spoken,     │
│  ...', 'position': 2}, {'title': 'What is Natural Language Processing? Definition and Examples', 'link':        │
│  'https://www.coursera.org/articles/natural-language-processing', 'snippet': 'Natural language processing       │
│  (NLP) is a form of artificial intelligence (AI) that allows computers to understand human language.',          │
│  'position': 3}, {'title': 'What is Natural Language Processing? A Guide to NLP - Viso Suite', 'link':          │
│  'https://viso.ai/deep-learning/natural-language-processing/', 'snippet': 'Natural language processing (NLP)    │
│  is a field of computer science and artificial intelligence concerned with the interactions between computers   │
│  and human ...', 'position': 4}, {'title': 'What Is Natural Language Processing? NLP Explained | Workday US',   │
│  'link': 'https://blog.workday.com/en-us/what-natural-language-processing-nlp-explained.html', 'snippet':       │
│  'Natural language processing (NLP) is machine learning technology that enables computers to understand,        │
│  interpret, and generate human language.', 'position': 5}, {'title': 'Unravelling the secrets of natural        │
│  language processing - ISO', 'link': 'https://www.iso.org/artificial-intelligence/natura...                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Thought: Thought: I need to analyze the gathered information and identify key aspects of NLP.                  │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Natural Language Processing 2023 summary\"}"                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'Natural Language Processing 2023 summary', 'type': 'search', 'num': 10, 'engine':  │
│  'google'}, 'organic': [{'title': '9 Natural Language Processing Trends in 2023 | StartUs Insights', 'link':    │
│  'https://www.startus-insights.com/innovators-guide/natural-language-processing-trends/', 'snippet': 'Top 8     │
│  Natural Language Processing Trends in 2023 · 1. Virtual Assistants · 2. Sentiment Analysis · 3. Multilingual   │
│  Language Models · 4. Named Entity Recognition · 5.', 'position': 1}, {'title': 'Natural language processing    │
│  capabilities and use cases - the 2023 ...', 'link':                                                            │
│  'https://fastdatascience.com/natural-language-processing/natural-language-processing-capabilities-and-use-cas  │
│  es/', 'snippet': 'In simple terms, NLP allows machines or computers to understand the human language. It       │
│  comprises algorithms which recognise human texts, and ...', 'position': 2}, {'title': 'Emerging Natural        │
│  Language Processing Technologies of 2023', 'link':                                                             │
│  'https://medium.com/@surushatechnology53/emerging-natural-language-processing-technologies-of-2023-8816718a3c  │
│  fb', 'snippet': 'From chatbots to language translation, NLP technologies have found applications in various    │
│  domains, revolutionizing industries and enhancing user experiences.', 'position': 3}, {'title': 'What Is NLP   │
│  (Natural Language Processing)? - IBM', 'link':                                                                 │
│  'https://www.ibm.com/think/topics/natural-language-processing', 'snippet': 'NLP enables computers and digital  │
│  devices to recognize, understand and generate text and speech by combining computational linguistics, the      │
│  rule ...', 'position': 4}, {'title': 'Our Guide to Natural Language Processing, an Introduction to NLP',       │
│  'link': 'https://kili-technology.com/data-labeling/nlp', 'snippet': 'The goal of NLU is to enable machines to  │
│  understand the meaning of human language by identifying the entities, concepts, relationships, and intents     │
│  expressed in ...', 'position': 5}, {'title': '2023 Natural Language Processing: Major Trends and               │
│  Innovations', 'link': 'htt...                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction       │
│  between computers and human languages. It enables machines to understand, interpret, and generate human        │
│  language using algorithms and models, combining computational linguistics with machine learning techniques.    │
│  NLP applications include chatbots, sentiment analysis, machine translation, and automated content creation,    │
│  with advancements seen in areas like multilingual models and deep learning. In 2023, it continues to evolve,   │
│  influencing various sectors and enhancing user experiences.                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
print(output.raw)

Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human languages. It enables machines to understand, interpret, and generate human language using algorithms and models, combining computational linguistics with machine learning techniques. NLP applications include chatbots, sentiment analysis, machine translation, and automated content creation, with advancements seen in areas like multilingual models and deep learning. In 2023, it continues to evolve, influencing various sectors and enhancing user experiences.


In [11]:
from langchain_openai import ChatOpenAI

nlp_task = Task(
    description="Understand the topic of Natural Language Processing and sumamrize it for me",
    # agent=data_analyst,  # commenting out the agent now, letting the manager decide
    expected_output='Give a correct response'
)

calculator_agent = Agent(
    role='Calculator',
    goal=f'You calculate things',
    backstory='You love math',
    verbose=True,
    allow_delegation=False,
    tools=[]
)

math_task = Task(
    description="Tell me what 123 * 34 is",
    expected_output='Calculate this'
)


crew = Crew(
    tasks=[math_task, nlp_task],
    agents=[calculator_agent, data_analyst],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model_name="gpt-4.1", temperature=0.7),  # required if you are using hierarhical
    verbose=True
)

output = crew.kickoff()

🚀 Crew: crew
├── 📋 Task: d355922a-d922-44de-be69-87c07cf042f5
│   Assigned to: Crew Manager
│   Status: ✅ Completed
│   └── 🔧 Using Delegate work to coworker (1)
└── 📋 Task: dfd5ac10-9498-4130-8c68-58aa17310310
    Status: Executing Task...
    ├── 🔧 Using Delegate work to coworker (2)
    ├── 🔧 Used Search the internet with Serper (1)
    └── 🔧 Used Search the internet with Serper (2)

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Natural Language Processing (NLP) Overview                                                                 │
│                                                                                                                 │
│  #### What is NLP?                                                                                              │
│  Natural Language Processing (NLP) is a branch of artificial intelligence (AI) that focuses on the interaction  │
│  between computers and humans through natural language. The goal of NLP is to enable machines to understand,    │
│  interpret, and generate human language in a meaningful way. By leveraging computational linguistics, NLP       │
│  combines rule-based modeling of human language with statistical learning methods to process and analyze large  │
│  amounts of natural language data.                                                                              │
│                                                                                                                 │
│  #### Main Goals of NLP                                                                                         │
│  The primary aims of NLP include:                                                                               │
│  1. **Understanding Human Language**: Enabling computers to grasp the semantic meaning of phrases, sentences,   │
│  and larger bodies of text as humans do.                                                                        │
│  2. **Language Generation**: Allowing machines to create coherent and contextually relevant sentences that can  │
│  communicate information effectively.                                                                           │
│  3. **Text Analysis**: Extracting insights, patterns, and information from unstructured data sources, such as   │
│  articles, reviews, or any written content.                                                                     │
│  4. **Speech Recognition and Processing**: Converting spoken language into written text, facilitating voice     │
│  command technologies and conversational agents.                                                                │
│                                                                                                                 │
│  #### Common Techniques Used in NLP                                                                             │
│  NLP employs various techniques, including:                                                                     │
│                                                                                                                 │
│  1. **Tokenization**: Breaking down text into smaller units, such as words or phrases, making it easier to      │
│  analyze.                                                                                                       │
│  2. **Part-of-Speech Tagging**: Identifying the role of each word (nouns, verbs, adjectives, etc.) within       │
│  sentences to understand their grammatical relationships.                                                       │
│  3. **Named Entity Recognition (NER)**: Detecting and classifying named entities (people, organizations,        │
│  locations) within text.                                                                                        │
│  4. **Sentiment Analysis**: Classifying text based on i

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: dfd5ac10-9498-4130-8c68-58aa17310310                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 17cf8f38-8a17-4101-9529-5f1d745d87e8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Natural Language Processing (NLP) Overview                                                   │
│                                                                                                                 │
│  #### What is NLP?                                                                                              │
│  Natural Language Processing (NLP) is a branch of artificial intelligence (AI) that focuses on the interaction  │
│  between computers and humans through natural language. The goal of NLP is to enable machines to understand,    │
│  interpret, and generate human language in a meaningful way. By leveraging computational linguistics, NLP       │
│  combines rule-based modeling of human language with statistical learning methods to process and analyze large  │
│  amounts of natural language data.                                                                              │
│                                                                                                                 │
│  #### Main Goals of NLP                                                                                         │
│  The primary aims of NLP include:                                                                               │
│  1. **Understanding Human Language**: Enabling computers to grasp the semantic meaning of phrases, sentences,   │
│  and larger bodies of text as humans do.                                                                        │
│  2. **Language Generation**: Allowing machines to create coherent and contextually relevant sentences that can  │
│  communicate information effectively.                                                                           │
│  3. **Text Analysis**: Extracting insights, patterns, and information from unstructured data sources, such as   │
│  articles, reviews, or any written content.                                                                     │
│  4. **Speech Recognition and Processing**: Converting spoken language into written text, facilitating voice     │
│  command technologies and conversational agents.                                                                │
│                                                                                                                 │
│  #### Common Techniques Used in NLP                                                                             │
│  NLP employs various techniques, including:                                                                     │
│                                                                                                                 │
│  1. **Tokenization**: Breaking down text into smaller units, such as words or phrases, making it easier to      │
│  analyze.                                                                                                       │
│  2. **Part-of-Speech Tagging**: Identifying the role of each word (nouns, verbs, adjectives, etc.) within       │
│  sentences to understand their grammatical relationships.                                                       │
│  3. **Named Entity Recognition (NER)**: Detecting and classifying named entities (people, organizations,        │
│  locations) within text.                              

In [12]:
for output in output.tasks_output:
    print(output)
    print('----')

The product of 123 and 34 is 4,182.
----
### Natural Language Processing (NLP) Overview

#### What is NLP?
Natural Language Processing (NLP) is a branch of artificial intelligence (AI) that focuses on the interaction between computers and humans through natural language. The goal of NLP is to enable machines to understand, interpret, and generate human language in a meaningful way. By leveraging computational linguistics, NLP combines rule-based modeling of human language with statistical learning methods to process and analyze large amounts of natural language data.

#### Main Goals of NLP
The primary aims of NLP include:
1. **Understanding Human Language**: Enabling computers to grasp the semantic meaning of phrases, sentences, and larger bodies of text as humans do.
2. **Language Generation**: Allowing machines to create coherent and contextually relevant sentences that can communicate information effectively.
3. **Text Analysis**: Extracting insights, patterns, and information from

In [13]:
output.raw

'### Natural Language Processing (NLP) Overview\n\n#### What is NLP?\nNatural Language Processing (NLP) is a branch of artificial intelligence (AI) that focuses on the interaction between computers and humans through natural language. The goal of NLP is to enable machines to understand, interpret, and generate human language in a meaningful way. By leveraging computational linguistics, NLP combines rule-based modeling of human language with statistical learning methods to process and analyze large amounts of natural language data.\n\n#### Main Goals of NLP\nThe primary aims of NLP include:\n1. **Understanding Human Language**: Enabling computers to grasp the semantic meaning of phrases, sentences, and larger bodies of text as humans do.\n2. **Language Generation**: Allowing machines to create coherent and contextually relevant sentences that can communicate information effectively.\n3. **Text Analysis**: Extracting insights, patterns, and information from unstructured data sources, suc

In [16]:
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool]
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

In [17]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [18]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)

In [19]:
# writes tries to use delegate to worker wrong. Fixes itself
result = crew.kickoff()

print("-----------------------------")
print(result)


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.                               │
│    Identify key trends, breakthrough technologies, and potential industry impacts.                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: I need to gather information on the latest advancements in AI for 2024, including key trends,         │
│  breakthrough technologies, and potential industry impacts.                                                     │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest advancements in AI in 2024 key trends breakthrough technologies industry impacts  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'latest advancements in AI in 2024 key trends breakthrough technologies industry    │
│  impacts', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '6 Game-Changing AI          │
│  Breakthroughs That Defined 2024 - Forbes', 'link':                                                             │
│  'https://www.forbes.com/sites/bernardmarr/2024/12/16/6-game-changing-ai-breakthroughs-that-defined-2024/',     │
│  'snippet': "From Apple's entry into generative AI to unprecedented achievements in robotics and art, 2024      │
│  marked a transformative year in artificial intelligence ...", 'position': 1}, {'title': 'Top 11 New            │
│  Technologies in AI: Exploring the Latest Trends - devabit', 'link':                                            │
│  'https://devabit.com/blog/top-11-new-technologies-in-ai-exploring-the-latest-trends/', 'snippet': 'From the    │
│  remarkable advancements in natural language processing (NLP) and GPT to the impact of AI on the sales and      │
│  marketing industries, we will explore the ...', 'position': 2}, {'title': 'McKinsey technology trends outlook  │
│  2025', 'link': 'https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/the-top-trends-in-tech',   │
│  'snippet': "From the rise of robotics and autonomous systems to the imperative for responsible AI              │
│  innovations, this year's technology developments ...", 'position': 3}, {'title': 'Artificial Intelligence      │
│  Breakthroughs: Key Developments to Expect in ...', 'link':                                                     │
│  'https://www.ironhack.com/us/blog/artificial-intelligence-breakthroughs-a-look-ahead-to-2024', 'snippet':      │
│  'Key AI Breakthroughs to Watch in 2025 · 1. Deep Learning Advancements · 2. Next-Gen Natural Language          │
│  Processing (NLP) · 3. Healthcare ...', 'position': 4}, {'title': 'Latest AI Breakthroughs and News: May,       │
│  June, July 2025', 'link': 'https://www.crescendo.ai/news/latest-ai-news-and-updates', 'snippet': "Wondering    │
│  what's happening in the AI world? Here are the latest AI breakthroughs and news that are shaping the world     │
│  around us!", 'position': 5}, {'title': 'The Top Artificial Intelligence Trends - IBM', 'link':                 │
│  'https://www.ibm.com/think/insights/...                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I have gathered relevant sources that outline the latest advancements in AI in 2024. I will  │
│  compile the insights from these articles into a comprehensive analysis.                                        │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I need to formulate a clear question for the Senior Research Analyst about the blog post     │
│  that needs to be written on AI advancements.                                                                   │
│                                                                                                                 │
│  Using Tool: Ask question to coworker                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"question\": \"Can you provide a comprehensive blog post that highlights the most significant AI advanceme  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Exploring AI Advancements for 2024: A New Horizon**                                                          │
│                                                                                                                 │
│  As we approach 2024, the landscape of artificial intelligence (AI) is set to evolve in unprecedented ways.     │
│  Key trends are surfacing, notably in areas such as Generative AI capabilities, ethical governance, and         │
│  applications in healthcare. As businesses and consumers alike lean increasingly into AI technologies, the      │
│  spotlight shines on Generative AI, utilizing deep learning for content creation that spans text, images, and   │
│  even music. This shift not only improves creative processes but also fosters new forms of interaction and      │
│  engagement with technology. Ethical governance of AI is paramount, with organizations prioritizing             │
│  transparency, accountability, and fairness to mitigate the risks of bias and misinformation. Recent            │
│  discussions highlight the urgent need for frameworks that ensure AI development aligns with human values and   │
│  societal norms.                                                                                                │
│                                                                                                                 │
│  In healthcare, the integration of AI continues to drive innovation, improving patient outcomes through         │
│  enhanced diagnostic tools, treatment predictions, and personalized medicine. AI systems are being employed to  │
│  analyze vast quantities of data, providing physicians with actionable insights that lead to timely decisions.  │
│  The 2024 AI Index Report emphasizes the rapid uptake of AI in health systems, showcasing an uptick in          │
│  AI-driven tools within diagnostic imaging and patient monitoring. This integration not only supports           │
│  healthcare professionals but also empowers patients through tailored health solutions, creating a more         │
│  interactive healthcare ecosystem.                                                                              │
│                                                                                                                 │
│  The democratization of AI tools is another promising trend, enabling individuals and small businesses to       │
│  harness advanced analytics and machine learning without requiring extensive technical expertise. With          │
│  platforms offering accessible AI capabilities, the barriers to entry are lowering significantly. As            │
│  industries embrace AI-powered automation, businesses are leveraging technologies to enhance operational        │
│  effici...                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  Exploring AI Advancements for 2024: A New Horizon                                                              │
│                                                                                                                 │
│  As we approach 2024, the landscape of artificial intelligence (AI) is set to evolve in unprecedented ways.     │
│  Key trends are surfacing, notably in areas such as Generative AI capabilities, ethical governance, and         │
│  applications in healthcare. As businesses and consumers alike lean increasingly into AI technologies, the      │
│  spotlight shines on Generative AI, utilizing deep learning for content creation that spans text, images, and   │
│  even music. This shift not only improves creative processes but also fosters new forms of interaction and      │
│  engagement with technology. Ethical governance of AI is paramount, with organizations prioritizing             │
│  transparency, accountability, and fairness to mitigate the risks of bias and misinformation. Recent            │
│  discussions highlight the urgent need for frameworks that ensure AI development aligns with human values and   │
│  societal norms.                                                                                                │
│                                                                                                                 │
│  In healthcare, the integration of AI continues to drive innovation, improving patient outcomes through         │
│  enhanced diagnostic tools, treatment predictions, and personalized medicine. AI systems are being employed to  │
│  analyze vast quantities of data, providing physicians with actionable insights that lead to timely decisions.  │
│  The 2024 AI Index Report emphasizes the rapid uptake of AI in health systems, showcasing an uptick in          │
│  AI-driven tools within diagnostic imaging and patient monitoring. This integration not only supports           │
│  healthcare professionals but also empowers patients through tailored health solutions, creating a more         │
│  interactive healthcare ecosystem.                                                                              │
│                                                                                                                 │
│  The democratization of AI tools is another promising trend, enabling individuals and small businesses to       │
│  harness advanced analytics and machine learning without requiring extensive technical expertise. With          │
│  platforms offering accessible AI capabilities, the barriers to entry are lowering significantly. As            │
│  industries embrace AI-powered automation, businesses are leveraging technologies to enhance operational        │
│  efficiency and reduce costs. From smart manufacturing to automated customer service, the trend is reshaping    │
│  job landscapes while necessitating new skills training in the workforce.                                       │
│                                                                                                                 │
│  Breakthrough technologies such as advanced AI models, ImageFX for visual content creation, and MusicFX for     │
│  generating audio are on the rise, signaling a shift to

-----------------------------
```
Exploring AI Advancements for 2024: A New Horizon

As we approach 2024, the landscape of artificial intelligence (AI) is set to evolve in unprecedented ways. Key trends are surfacing, notably in areas such as Generative AI capabilities, ethical governance, and applications in healthcare. As businesses and consumers alike lean increasingly into AI technologies, the spotlight shines on Generative AI, utilizing deep learning for content creation that spans text, images, and even music. This shift not only improves creative processes but also fosters new forms of interaction and engagement with technology. Ethical governance of AI is paramount, with organizations prioritizing transparency, accountability, and fairness to mitigate the risks of bias and misinformation. Recent discussions highlight the urgent need for frameworks that ensure AI development aligns with human values and societal norms.

In healthcare, the integration of AI continues to drive inn

In [20]:
writer.allow_delegation = False  # turn off delegation, resulting in all hard-coded delegation

In [21]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [22]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)
result = crew.kickoff()
print("-----------------------------")
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.                               │
│    Identify key trends, breakthrough technologies, and potential industry impacts.                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: I need to search for the latest advancements in AI for 2024 to gather relevant information and        │
│  insights.                                                                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest advancements in AI 2024\"}"                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'latest advancements in AI 2024', 'type': 'search', 'num': 10, 'engine':            │
│  'google'}, 'organic': [{'title': '2024: A year of extraordinary progress and advancement in AI', 'link':       │
│  'https://blog.google/technology/ai/2024-ai-extraordinary-progress-advancement/', 'snippet': 'At the start of   │
│  2024, we introduced ImageFX, a new generative AI tool that creates images from text prompts, and MusicFX, a    │
│  tool for creating up-to-70-second audio clips also based on text prompts . At I/O, we shared an early preview  │
│  of MusicFX DJ, a tool that helps bring the joy of live music creation to more people.', 'position': 1},        │
│  {'title': '6 Game-Changing AI Breakthroughs That Defined 2024 - Forbes', 'link':                               │
│  'https://www.forbes.com/sites/bernardmarr/2024/12/16/6-game-changing-ai-breakthroughs-that-defined-2024/',     │
│  'snippet': "From Apple's entry into generative AI to unprecedented achievements in robotics and art, 2024      │
│  marked a transformative year in artificial intelligence ...", 'position': 2}, {'title': 'The State of AI:      │
│  Global survey - McKinsey', 'link':                                                                             │
│  'https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai', 'snippet': 'Use of          │
│  generative AI increased from 33% in 2023 to 71% in 2024. Use of AI by business function for the latest data    │
│  varied from 36% in IT to 12% ...', 'position': 3}, {'title': 'Generative AI Developments & Trends in 2024: A   │
│  Timeline', 'link':                                                                                             │
│  'https://www.channelinsider.com/managed-services/generative-ai-developments-trends-year-in-review/',           │
│  'snippet': 'Discover key milestones and advancements in generative AI for 2024, highlighting innovations and   │
│  trends that will shape the future of artificial intelligence.', 'position': 4, 'sitelinks': [{'title': 'What   │
│  is GenAI?', 'link':                                                                                            │
│  'https://www.channelinsider.com/managed-services/generative-ai-developments-trends-year-in-review/#what-is-ge  │
│  nai'}, {'title': 'timeline of GenAI...', 'link':                                                               │
│  'https://www.channelinsider.com/managed-services/generative-ai-developments-trends-year-in-...                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - **Generative AI Tools**: Introduction of innovative generative AI tools:                                     │
│    - **ImageFX**: A generative AI tool for creating images from text prompts.                                   │
│    - **MusicFX**: A tool for generating short audio clips based on text, enhancing user engagement in music     │
│  creation.                                                                                                      │
│                                                                                                                 │
│  - **Increased Adoption of Generative AI**:                                                                     │
│    - The use of generative AI surged from 33% in 2023 to 71% in 2024, highlighting its growing integration      │
│  across various industries.                                                                                     │
│                                                                                                                 │
│  - **Key Breakthroughs**:                                                                                       │
│    - Apple's significant leap into generative AI, showcasing the technology's versatility and expanding its     │
│  reach.                                                                                                         │
│    - Advancements in robotics and AI creating art have marked the year as transformative for the AI landscape.  │
│                                                                                                                 │
│  - **Technological Innovations**:                                                                               │
│    - Emergence of advanced biometric technologies, including:                                                   │
│      - Gait recognition                                                                                         │
│      - DNA-based identification                                                                                 │
│      - Brainwave authentication                                                                                 │
│                                                                                                                 │
│  - **AI Trends Identified**:                                                                                    │
│    - Increasing emphasis on ethics and transparency in AI deployment, with regulatory frameworks evolving.      │
│    - A focus on sustainable AI practices to support environmental initiatives.                                  │
│                                                                                                                 │
│  - **Notable AI Stories**:                                                                                      │
│    - Significant impacts from AI on cybersecurity, including advanced threat detection and response systems.    │
│    - Noteworthy legislation affecting AI usage in industries, focusing on data privacy and protection.          │
│    - Major developments in AI hardware that improve computational power and efficiency.                         │
│                                                                                                                 │
│  - **Collaborative AI**:                               

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Task: Using the insights provided, develop an engaging blog                                                    │
│    post that highlights the most significant AI advancements.                                                   │
│    Your post should be informative yet accessible, catering to a tech-savvy audience.                           │
│    Make it sound cool, avoid complex words so it doesn't sound like AI.                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **The Rise of Generative AI: A Year of Transformation and Innovation**                                         │
│                                                                                                                 │
│  As we step into 2024, the world of technology is buzzing with excitement, particularly in the realm of         │
│  Artificial Intelligence (AI). The landscape has undergone significant evolution, particularly around           │
│  generative AI tools that are reshaping how we approach creativity and innovation. Among the most notable       │
│  advancements are tools like **ImageFX**, which allows users to create stunning visuals from mere text          │
│  prompts, and **MusicFX**, a pioneering program that crafts short audio clips tailored to specific textual      │
│  input. These tools are not only making creative processes more engaging but are also opening doors for         │
│  individuals to explore their artistic sides with unprecedented ease.                                           │
│                                                                                                                 │
│  What’s remarkable about the surge of generative AI is the adoption rate, which skyrocketed from 33% in 2023    │
│  to a staggering 71% in just one year. This phenomenal growth signifies a cultural shift, where industries      │
│  across the board—from entertainment to education—are integrating AI-driven solutions into their workflows.     │
│  Companies like Apple have made significant strides in adopting generative AI, showcasing the technology's      │
│  versatility. Meanwhile, advancements in robotics have allowed AI to not only assist in tasks but to also       │
│  create remarkable pieces of artwork, marking 2024 as a pivotal year for creativity powered by technology.      │
│                                                                                                                 │
│  Yet, technological innovations are not confined to generative art. The development of **advanced biometric     │
│  technologies**, including gait recognition, DNA-based identification, and brainwave authentication, mark a     │
│  shift towards a more secure digital age. These advancements promise to revolutionize how we authenticate our   │
│  identities, potentially making passwords a thing of the past. Meanwhile, the conversation around AI is         │
│  evolving to prioritize ethics and transparency. As organizations recognize the need for responsible AI         │
│  deployment, we're witnessing the emergence of regulatory frameworks designed to protect data privacy and       │
│  ensure that AI technologies are developed and utilized in a sustainable manner.                                │
│                                                                                                                 │
│  Moreover, the impact of AI extends deep into industries such as healthcare, finance, and education, where it   │
│  drives insights, enhances predictive analytics, and offers personalized experiences. The collaborative         │
│  frameworks that have developed between human intelligence and AI systems are fostering better synergy, making  │
│  it an exciting time for professionals across various fields. As we look ahead, the foundation laid in 2024     │
│  promises a continuous wave of innovations, with genera

-----------------------------
**The Rise of Generative AI: A Year of Transformation and Innovation**

As we step into 2024, the world of technology is buzzing with excitement, particularly in the realm of Artificial Intelligence (AI). The landscape has undergone significant evolution, particularly around generative AI tools that are reshaping how we approach creativity and innovation. Among the most notable advancements are tools like **ImageFX**, which allows users to create stunning visuals from mere text prompts, and **MusicFX**, a pioneering program that crafts short audio clips tailored to specific textual input. These tools are not only making creative processes more engaging but are also opening doors for individuals to explore their artistic sides with unprecedented ease.

What’s remarkable about the surge of generative AI is the adoption rate, which skyrocketed from 33% in 2023 to a staggering 71% in just one year. This phenomenal growth signifies a cultural shift, where indus

In [23]:
print(task1.output.raw)

- **Generative AI Tools**: Introduction of innovative generative AI tools:
  - **ImageFX**: A generative AI tool for creating images from text prompts.
  - **MusicFX**: A tool for generating short audio clips based on text, enhancing user engagement in music creation.
  
- **Increased Adoption of Generative AI**: 
  - The use of generative AI surged from 33% in 2023 to 71% in 2024, highlighting its growing integration across various industries.

- **Key Breakthroughs**:
  - Apple's significant leap into generative AI, showcasing the technology's versatility and expanding its reach.
  - Advancements in robotics and AI creating art have marked the year as transformative for the AI landscape.
  
- **Technological Innovations**: 
  - Emergence of advanced biometric technologies, including:
    - Gait recognition
    - DNA-based identification
    - Brainwave authentication
   
- **AI Trends Identified**:
  - Increasing emphasis on ethics and transparency in AI deployment, with regulatory f

In [24]:
print(task2.output.raw)

**The Rise of Generative AI: A Year of Transformation and Innovation**

As we step into 2024, the world of technology is buzzing with excitement, particularly in the realm of Artificial Intelligence (AI). The landscape has undergone significant evolution, particularly around generative AI tools that are reshaping how we approach creativity and innovation. Among the most notable advancements are tools like **ImageFX**, which allows users to create stunning visuals from mere text prompts, and **MusicFX**, a pioneering program that crafts short audio clips tailored to specific textual input. These tools are not only making creative processes more engaging but are also opening doors for individuals to explore their artistic sides with unprecedented ease.

What’s remarkable about the surge of generative AI is the adoption rate, which skyrocketed from 33% in 2023 to a staggering 71% in just one year. This phenomenal growth signifies a cultural shift, where industries across the board—from en

# Building a Search Agent with a Custom Tool

In [ ]:
os.environ["SERP_API_KEY"] = 'your_serp_key'

In [31]:
import json
import os
import requests
from crewai import Agent, Task
from serpapi import GoogleSearch
from crewai.tools import BaseTool

class SearchTool(BaseTool):
    name: str = "Google Search"
    description: str = """Useful to search the internet about a given topic and return relevant results."""

    def _run(self, query: str, top_k: int = 3) -> str:
        params: dict = {
            "engine": "google",
            "google_domain": "google.com",
            "gl": "us",
            "hl": "en",
            "q": query,
            "api_key": os.environ["SERP_API_KEY"],
        }

        search = GoogleSearch(params)
        response = search.get_dict()
        # check if there is an organic results key, don't include sponsor results
        if 'organic_results' not in response:
            return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
        else:
            results = response['organic_results']
            string = []
            for result in results[:top_k]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}", f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}", "\n-----------------"
                    ]))
                except KeyError:
                    next

            return '\n'.join(string)

In [32]:
SearchTool().description # where I specifically describe how to use the tool

"Tool Name: Google Search\nTool Arguments: {'query': {'description': None, 'type': 'str'}, 'top_k': {'description': None, 'type': 'int'}}\nTool Description: Useful to search the internet about a given topic and return relevant results."

In [33]:
print(SearchTool().run(query="Who is Sinan Ozdemir?"))

Using Tool: Google Search
Title: Sinan Ozdemir - LoopGenius
Link: https://www.linkedin.com/in/sinan-ozdemir
Snippet: Specialized in areas including time management as well as mathematics and computer science logistics.

-----------------
Title: Sinan Ozdemir
Link: https://sinanozdemir.ai/
Snippet: Helping companies leverage AI technology to solve complex problems. Founder, author, and consultant specializing in AI, LLMs, and data science.

-----------------
Title: Sinan Ozdemir
Link: https://odsc.com/blog/speaker/sinan-ozdemir/
Snippet: Sinan Ozdemir is a mathematician, data scientist, NLP expert, lecturer, and accomplished author. He is currently applying my extensive knowledge and experience ...

-----------------


In [34]:
researcher_agent = Agent(
    role='Researcher',
    goal='Look up information on the internet and scrape the websites for relevant information',
    backstory="""You have a knack for dissecting complex data and presenting actionable insights.""",
    tools=[
        SearchTool()
    ],
    verbose=True
)
email_agent = Agent(
    role='Emailer',
    goal='Write short, warm emails to people',
    backstory="""You are an assistant.""",
    tools=[],
    verbose=True
)

sample_lookup_task = Task(
  description="""Lookup Sinan Ozdemir""",
  expected_output="A lookup report of a query",
  agent=researcher_agent
)

email_task = Task(
  description="""Write an email to Sinan Ozdemir asking him to get coffee. Personalize it""",
  expected_output="A short warm email",
  agent=email_agent
)

crew = Crew(
    tasks=[sample_lookup_task, email_task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

🚀 Crew: crew
├── 📋 Task: 42c046e8-0aa5-4320-9605-b07182a25b43
│   Assigned to: Researcher
│   Status: ✅ Completed
│   └── 🔧 Used Google Search (1)
└── 📋 Task: 36e89a69-3ee9-49bf-bad0-203e5d7781e7
    Status: Executing Task...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Emailer                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Coffee Chat?                                                                                          │
│                                                                                                                 │
│  Hi Sinan,                                                                                                      │
│                                                                                                                 │
│  I hope this message finds you well! I’ve been following your impressive work in AI and data science,           │
│  especially your recent book on LLMs. As someone deeply interested in the intersections of technology and       │
│  problem-solving, I’m eager to hear more about your insights and experiences.                                   │
│                                                                                                                 │
│  Would you be open to grabbing a coffee sometime? I’d love the chance to connect and learn from your            │
│  expertise.                                                                                                     │
│                                                                                                                 │
│  Looking forward to hearing from you!                                                                           │
│                                                                                                                 │
│  Best,                                                                                                          │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 36e89a69-3ee9-49bf-bad0-203e5d7781e7                                                                     │
│  Agent: Emailer                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ba37748-7b7d-4a4d-b4d1-2c5f5f2723e6                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Coffee Chat?                                                                            │
│                                                                                                                 │
│  Hi Sinan,                                                                                                      │
│                                                                                                                 │
│  I hope this message finds you well! I’ve been following your impressive work in AI and data science,           │
│  especially your recent book on LLMs. As someone deeply interested in the intersections of technology and       │
│  problem-solving, I’m eager to hear more about your insights and experiences.                                   │
│                                                                                                                 │
│  Would you be open to grabbing a coffee sometime? I’d love the chance to connect and learn from your            │
│  expertise.                                                                                                     │
│                                                                                                                 │
│  Looking forward to hearing from you!                                                                           │
│                                                                                                                 │
│  Best,                                                                                                          │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [35]:
print(result)

Subject: Coffee Chat?  

Hi Sinan,  

I hope this message finds you well! I’ve been following your impressive work in AI and data science, especially your recent book on LLMs. As someone deeply interested in the intersections of technology and problem-solving, I’m eager to hear more about your insights and experiences.  

Would you be open to grabbing a coffee sometime? I’d love the chance to connect and learn from your expertise.  

Looking forward to hearing from you!  

Best,  
[Your Name]


## Trip Advisor

In [40]:
from crewai import Agent

class TripAgents():

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide 2-3 insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTool(),
            web_rag_tool
        ],
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with
        decades of experience""",
        tools=[],
        verbose=True)

In [41]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasks():

  def gather_task(self, agent, origin, city, interests, range):
    return Task(description= dedent(f"""
        I want you to create a comprehensive city guide.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, and must-visit landmarks.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
        """),

            expected_output=dedent(f"""
        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.

        """),
                agent=agent)

  def plan_task(self, agent, origin, city, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 1-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
      """),
            expected_output=dedent(f"""
        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special!
        """),
                agent=agent)

In [42]:
class TripCrew:

  def __init__(self, origin, city, date_range, interests):
    self.city = city
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    gather_task = tasks.gather_task(
      agents.local_expert(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      agents.travel_concierge(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )

    # Initiate Crew
    crew = Crew(
      tasks=[gather_task, plan_task],
      process=Process.sequential,
      verbose=True
    )

    result = crew.kickoff()
    return result

In [43]:
trip_crew = TripCrew('SF', "Paris", 'December 2025', 'art')
result = trip_crew.run()

🚀 Crew: crew
├── 📋 Task: 86d4e7e9-d0ae-44e0-ba60-398097b1158c
│   Assigned to: Local Expert at this city
│   Status: ✅ Completed
│   ├── 🔧 Used Google Search (2)
│   └── 🔧 Used Search in a specific website (1)
└── 📋 Task: 77514996-8dc0-416f-aa14-a43634e9e399
    Status: Executing Task...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Amazing Travel Concierge                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # **Amazing 1-Day Travel Itinerary for Paris (December 2025)**                                                 │
│                                                                                                                 │
│  ### **Introduction**                                                                                           │
│  Welcome to your one-day artistic adventure in Paris! This guide is tailored for art enthusiasts like you,      │
│  offering the best experiences the city has to offer in a single day. From famed museums to local               │
│  neighborhoods bursting with creativity, let’s make your trip unforgettable!                                    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Weather Forecast**                                                                                       │
│  **Date:** December 15, 2025                                                                                    │
│  **Expected Weather:** High 7°C (44°F) | Low 3°C (37°F)                                                         │
│  **Conditions:** Mostly sunny with intermittent clouds.                                                         │
│                                                                                                                 │
│  **Clothing Suggestions:**                                                                                      │
│  - Warm layers (thermal base layers, sweaters)                                                                  │
│  - Waterproof jacket or coat                                                                                    │
│  - Comfortable walking shoes                                                                                    │
│  - Scarf, gloves, and a hat                                                                                     │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Itinerary Schedule**                                                                                     │
│                                                                                                                 │
│  #### **8:00 AM - Breakfast at Café de Flore**                                                                  │
│  Start your day with iconic Parisian breakfast at **Café de Flore** (172 Boulevard Saint-Germain). This         │
│  historic café has been a favorite among artists and intellectuals like Jean-Paul Sartre and Simone de          │
│  Beauvoir. Enjoy a traditional croissant with rich coffee while soaking in the atmosphere.                      │
│                                                                                                                 │
│  **Cost:** Approximately €15                           

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 77514996-8dc0-416f-aa14-a43634e9e399                                                                     │
│  Agent: Amazing Travel Concierge                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2ade5148-3bc4-4c11-9cf0-886913cb1aea                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # **Amazing 1-Day Travel Itinerary for Paris (December 2025)**                                   │
│                                                                                                                 │
│  ### **Introduction**                                                                                           │
│  Welcome to your one-day artistic adventure in Paris! This guide is tailored for art enthusiasts like you,      │
│  offering the best experiences the city has to offer in a single day. From famed museums to local               │
│  neighborhoods bursting with creativity, let’s make your trip unforgettable!                                    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Weather Forecast**                                                                                       │
│  **Date:** December 15, 2025                                                                                    │
│  **Expected Weather:** High 7°C (44°F) | Low 3°C (37°F)                                                         │
│  **Conditions:** Mostly sunny with intermittent clouds.                                                         │
│                                                                                                                 │
│  **Clothing Suggestions:**                                                                                      │
│  - Warm layers (thermal base layers, sweaters)                                                                  │
│  - Waterproof jacket or coat                                                                                    │
│  - Comfortable walking shoes                                                                                    │
│  - Scarf, gloves, and a hat                                                                                     │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Itinerary Schedule**                                                                                     │
│                                                                                                                 │
│  #### **8:00 AM - Breakfast at Café de Flore**                                                                  │
│  Start your day with iconic Parisian breakfast at **Café de Flore** (172 Boulevard Saint-Germain). This         │
│  historic café has been a favorite among artists and intellectuals like Jean-Paul Sartre and Simone de          │
│  Beauvoir. Enjoy a traditional croissant with rich coffee while soaking in the atmosphere.                      │
│                                                       

In [44]:
print(result)

# **Amazing 1-Day Travel Itinerary for Paris (December 2025)**

### **Introduction**
Welcome to your one-day artistic adventure in Paris! This guide is tailored for art enthusiasts like you, offering the best experiences the city has to offer in a single day. From famed museums to local neighborhoods bursting with creativity, let’s make your trip unforgettable!

---

### **Weather Forecast**
**Date:** December 15, 2025  
**Expected Weather:** High 7°C (44°F) | Low 3°C (37°F)  
**Conditions:** Mostly sunny with intermittent clouds.  

**Clothing Suggestions:**
- Warm layers (thermal base layers, sweaters)
- Waterproof jacket or coat
- Comfortable walking shoes
- Scarf, gloves, and a hat

---

### **Itinerary Schedule**

#### **8:00 AM - Breakfast at Café de Flore**
Start your day with iconic Parisian breakfast at **Café de Flore** (172 Boulevard Saint-Germain). This historic café has been a favorite among artists and intellectuals like Jean-Paul Sartre and Simone de Beauvoir. Enjoy a tr

In [45]:
print("## Welcome to Trip Planner Crew")
print('-——————————')
location = input(
dedent("""
  From where will you be traveling from?
"""))
city = input(
dedent("""
  Where do you want to go?
"""))
date_range = input(
dedent("""
  What is the date range you are interested in traveling?
"""))
interests = input(
dedent("""
  What are some of your high level interests and hobbies?
"""))

trip_crew = TripCrew(location, city, date_range, interests)
result = trip_crew.run()

🚀 Crew: crew
├── 📋 Task: 328d382a-71a4-491c-9368-b7f1ae5bfe8e
│   Assigned to: Local Expert at this city
│   Status: ✅ Completed
│   ├── 🔧 Used Google Search (3)
│   └── 🔧 Used Google Search (4)
└── 📋 Task: 988a16ea-b286-42e7-b55d-3e7c3044e494
    Status: Executing Task...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Amazing Travel Concierge                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Berlin 1-Day Itinerary (October 2025)                                                                        │
│                                                                                                                 │
│  ## Overview                                                                                                    │
│  - **Trip Date**: October 2025                                                                                  │
│  - **From**: Princeton, NJ                                                                                      │
│  - **To**: Berlin, Germany                                                                                      │
│  - **Interests**: Beer, museums, trains                                                                         │
│                                                                                                                 │
│  ### Weather Forecast                                                                                           │
│  In October, Berlin experiences mild fall temperatures, averaging between 50°F to 60°F (10°C to 15°C). Expect   │
│  occasional rain, so prepare for both sunny days and light showers.                                             │
│                                                                                                                 │
│  ### Packing Suggestions                                                                                        │
│  - **Clothing**: Layers are essential. Pack a light jacket, comfortable walking shoes, jeans, and warm          │
│  sweaters. A compact umbrella and a scarf can keep you warm.                                                    │
│  - **Items**: Portable charger, travel guide, U-Bahn map, reusable water bottle, and a notebook for notes.      │
│                                                                                                                 │
│  ## Detailed Itinerary                                                                                          │
│                                                                                                                 │
│  ### Morning                                                                                                    │
│  #### 8:00 AM - Breakfast at **House of Small Wonder**                                                          │
│  Start your day at the cozy **House of Small Wonder**, renowned for its eco-friendly ambiance and delicious     │
│  breakfast options. Enjoy a hearty meal of the famous Japanese-style breakfast bowl with a coffee or matcha     │
│  latte. **(Estimated Cost: $15/person)**                                                                        │
│                                                                                                                 │
│  #### 9:00 AM - Visit **Brandenburg Gate**                                                                      │
│  After breakfast, head to the iconic **Brandenburg Gate** (20-minute walk). As you stand in front of this       │
│  historical symbol of unity, take some time to capture stunning photos and appreciate the beauty of the         │
│  surrounding gardens.                                                                                           │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 988a16ea-b286-42e7-b55d-3e7c3044e494                                                                     │
│  Agent: Amazing Travel Concierge                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1e3e0c52-1730-470e-9f80-08ccbe533519                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Berlin 1-Day Itinerary (October 2025)                                                          │
│                                                                                                                 │
│  ## Overview                                                                                                    │
│  - **Trip Date**: October 2025                                                                                  │
│  - **From**: Princeton, NJ                                                                                      │
│  - **To**: Berlin, Germany                                                                                      │
│  - **Interests**: Beer, museums, trains                                                                         │
│                                                                                                                 │
│  ### Weather Forecast                                                                                           │
│  In October, Berlin experiences mild fall temperatures, averaging between 50°F to 60°F (10°C to 15°C). Expect   │
│  occasional rain, so prepare for both sunny days and light showers.                                             │
│                                                                                                                 │
│  ### Packing Suggestions                                                                                        │
│  - **Clothing**: Layers are essential. Pack a light jacket, comfortable walking shoes, jeans, and warm          │
│  sweaters. A compact umbrella and a scarf can keep you warm.                                                    │
│  - **Items**: Portable charger, travel guide, U-Bahn map, reusable water bottle, and a notebook for notes.      │
│                                                                                                                 │
│  ## Detailed Itinerary                                                                                          │
│                                                                                                                 │
│  ### Morning                                                                                                    │
│  #### 8:00 AM - Breakfast at **House of Small Wonder**                                                          │
│  Start your day at the cozy **House of Small Wonder**, renowned for its eco-friendly ambiance and delicious     │
│  breakfast options. Enjoy a hearty meal of the famous Japanese-style breakfast bowl with a coffee or matcha     │
│  latte. **(Estimated Cost: $15/person)**                                                                        │
│                                                                                                                 │
│  #### 9:00 AM - Visit **Brandenburg Gate**                                                                      │
│  After breakfast, head to the iconic **Brandenburg Gate** (20-minute walk). As you stand in front of this       │
│  historical symbol of unity, take some time to capture stunning photos and appreciate the beauty of the         │
│  surrounding gardens.                                 

In [46]:
print("\n\n########################")
print("## Here is your Trip Plan")
print("########################\n")
print(result)



########################
## Here is your Trip Plan
########################

# Berlin 1-Day Itinerary (October 2025)

## Overview
- **Trip Date**: October 2025
- **From**: Princeton, NJ
- **To**: Berlin, Germany
- **Interests**: Beer, museums, trains

### Weather Forecast
In October, Berlin experiences mild fall temperatures, averaging between 50°F to 60°F (10°C to 15°C). Expect occasional rain, so prepare for both sunny days and light showers. 

### Packing Suggestions
- **Clothing**: Layers are essential. Pack a light jacket, comfortable walking shoes, jeans, and warm sweaters. A compact umbrella and a scarf can keep you warm.
- **Items**: Portable charger, travel guide, U-Bahn map, reusable water bottle, and a notebook for notes.

## Detailed Itinerary

### Morning
#### 8:00 AM - Breakfast at **House of Small Wonder**
Start your day at the cozy **House of Small Wonder**, renowned for its eco-friendly ambiance and delicious breakfast options. Enjoy a hearty meal of the famous Japan